In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx
import csv
from tqdm import tqdm
import json
import pandas as pd

In [2]:
def read_snapshot(filename, it):
    G = nx.Graph()
    with open(filename, 'r') as runfile:
        its = json.load(runfile)
        edgelist = its[it]['edges']
        opinions = its[it]['status']
        for edge in edgelist:
            G.add_node(int(edge[0])) #superhero in first column
            G.add_node(int(edge[1])) #superhero in second column
            G.add_edge(int(edge[0]), int(edge[1]), weight = 1)
        nx.set_node_attributes(G, {int(k): v for k,v in opinions.items()}, name="opinion")
        return G

def compute_ncc(graph):
    return nx.number_connected_components(graph)

def compute_avgdeg(graph):
    degree = dict(graph.degree())
    s = sum(degree.values())
    return s/graph.number_of_nodes()
    

def compute_triangles(graph):
    d = dict(nx.triangles(graph))
    s = sum(d.values())
    return s/graph.number_of_nodes()

def compute_clustering(graph):
    return nx.average_clustering(graph)

def compute_nac(graph):
    return nx.degree_assortativity_coefficient(graph)

def compute_assortativity(graph):
    
    def homophily(graph, v):
        neighbors = list(graph.neighbors(v))
        if len(neighbors) > 0:
            neighborsops = [graph.nodes[u]['opinion'] for u in neighbors]
            h =  sum([(((-2/max(graph.nodes[v]['opinion'], 1-graph.nodes[v]['opinion']))*(abs(graph.nodes[v]['opinion']-op)))+1) for op in neighborsops])/len(neighborsops)
            return h
        else:
            return 0
    
    hlist = []
    
    for v in list(G.nodes()):
        hv = homophily(graph, v)
        hlist.append(hv)
    
    return sum(hlist)/len(hlist)

In [3]:
%cd res/

/data1/users/pansanella/GitHub/AdaptiveAlgBiasOnSC/res


In [9]:
%ls

 README.txt  'rewiring er0.1 metrics.csv'


In [4]:
modelname = "rewiring"

In [5]:
mi = 1000000

In [6]:
import os
aggrfile = open(f"{modelname} er0.1 metrics.csv", "w+")
header = "model,graph,n,p,pr,eps,gam,max_it\n"
aggrfile.write(header)
aggrfile.close()
aggrfile = open(f"{modelname} er0.1 metrics.csv", "a+")
for graph in ['er']:
    if graph == "er":
        p = 0.1
    else:
        p=5
    for pr in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for eps in [0.35]:
            for gam in [0.0, 0.5, 1.0, 1.5]:
                name = f"{modelname} {graph}{p} pr{pr} e{eps} g{gam} mi1000000"
                s = f"{modelname},{graph},100,{p},{pr},{eps},{gam},1000000\n"
                aggrfile.write(s)
aggrfile.close()
results = pd.read_csv(f"{modelname} er0.1 metrics.csv")

In [8]:
modelname = "rewiring"
for index, row in (results.iterrows()):
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    graph = row['graph']
    p = row['p']
    mi = row['max_it']
    ncc = []
    avgd = []
    tri = []
    cl = []
    ass = []
    avgh = []
    for nr in tqdm(range(30)):
        name = f"{modelname} er0.1 pr{pr} e{eps} g{gam} mi{mi}"    
        with open(f"{name} nr{nr}.json", 'r') as runfile:
            data = json.load(runfile)
            it = 1
            G = read_snapshot(f"{name} nr{nr}.json", it)
            ncc.append(compute_ncc(G))
            avgd.append(compute_avgdeg(G))
            tri.append(compute_triangles(G))
            cl.append(compute_clustering(G))
            ass.append(compute_nac(G))
            avgh.append(compute_assortativity(G))    
    row["avgncc_initial"] = sum(ncc)/len(ncc)
    row["avgavgd_initial"] = sum(avgd)/len(avgd)
    row["avgtri_initial"] = sum(tri)/len(tri)
    row['avgcl_initial'] = sum(cl)/len(cl)
    row['avgass_initial'] = sum(ass)/len(ass)
    row['avghom_initial'] = sum(avgh)/len(avgh)
    break

results
    # ncc = []
    # avgd = []
    # tri = []
    # cl = []
    # ass = []
    # avgh = []
    # for nr in tqdm(range(30)):
    #     name = f"{modelname} er0.1 pr{pr} e{eps} g{gam} mi{mi}"    
    #     with open(f"{name} nr{nr}.json", 'r') as runfile:
    #         data = json.load(runfile)
    #         for it in [len(data)-1]:
    #             G = read_snapshot(f"{name} nr{nr}.json", it)
    #             ncc.append(compute_ncc(G))
    #             avgd.append(compute_avgdeg(G))
    #             tri.append(compute_triangles(G))
    #             cl.append(compute_clustering(G))
    #             ass.append(compute_nac(G))
    #             avgh.append(compute_assortativity(G))
    # row["avgncc_final"] = sum(ncc)/len(ncc)
    # row["avgavgd_final"] = sum(avgd)/len(avgd)
    # row["avgtri_final"] = sum(tri)/len(tri)
    # row['avgcl_final'] = sum(cl)/len(cl)
    # row['avgass_final'] = sum(ass)/len(ass)
    # row['avghom_final'] = sum(avgh)/len(avgh)    

  0%|          | 0/30 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'rewiring er0.1 pr0.0 e0.35 g0.0 mi1000000 nr0.json'

In [ ]:
results.to_csv("tmp.csv")

In [ ]:
df = pd.read_csv("tmp.csv")

In [ ]:
df